In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta, timezone

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor

In [4]:
import os
from dotenv import load_dotenv
load_dotenv('./../01_python/.env')
SERVICE_KEY = os.getenv('RAIN_ID') # Decoding 키/Encoding 키 중 안내에 맞춰 사용
CSV_PATH = "SUWON_S_DATA_TABLE_GENDER_SUM.csv"  # 업로드한 파일 경로로 바꿔도 됨
NX, NY = 61, 121  # 수원 예시(정확한 위치 격자좌표로 수정 권장)
# 기상청 초단기실황(공공데이터포털) 엔드포인트
ULTRA_NCST_URL = "https://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst"
KST = timezone(timedelta(hours=9))

In [5]:
# -------------------------
# 1) 날짜 처리: 요일코드(01~07) 만들기 (데이터 정의와 동일)
#    월=01 ... 일=07
# -------------------------
def to_day_code(dt: datetime) -> int:
    # python weekday: 월=0 ... 일=6
    return int(dt.weekday() + 1)  # 1~7

In [6]:
# -------------------------
# 2) 초단기실황 base_date/base_time 계산
#    초단기실황은 "가장 최근 시각"으로 조회해야 함.
#    (단순 안전 로직: 현재 KST 기준 40분 전이면 직전 정시 사용)
# -------------------------
def latest_ultra_ncst_base(now_kst: datetime):
    # 예: 15:20이면 14:00 기준으로 조회(대개 갱신 지연 고려)
    # 예: 15:50이면 15:00 기준으로 조회
    if now_kst.minute < 40:
        base_dt = now_kst.replace(minute=0, second=0, microsecond=0) - timedelta(hours=1)
    else:
        base_dt = now_kst.replace(minute=0, second=0, microsecond=0)
    base_date = base_dt.strftime("%Y%m%d")
    base_time = base_dt.strftime("%H%M")  # HH00 형태
    return base_date, base_time


In [7]:
# -------------------------
# 3) 기상청 초단기실황 호출 -> (T1H:기온, RN1:1시간강수)
# -------------------------
def fetch_today_weather_ultra_ncst(nx: int, ny: int, service_key: str):
    now = datetime.now(KST)
    base_date, base_time = latest_ultra_ncst_base(now)

    params = {
        "serviceKey": service_key,
        "numOfRows": 1000,
        "pageNo": 1,
        "dataType": "JSON",
        "base_date": base_date,
        "base_time": base_time,
        "nx": nx,
        "ny": ny,
    }
    r = requests.get(ULTRA_NCST_URL, params=params, timeout=20)
    r.raise_for_status()
    data = r.json()

    items = data["response"]["body"]["items"]["item"]

    # 필요한 카테고리: T1H(기온), RN1(1시간강수) / 값은 문자열일 수 있음
    t1h = None
    rn1 = None
    for it in items:
        if it["category"] == "T1H":
            t1h = float(it["obsrValue"])
        elif it["category"] == "RN1":
            # RN1은 "강수없음" 같은 문자열이 올 수 있어 방어
            try:
                rn1 = float(it["obsrValue"])
            except:
                rn1 = 0.0

    if t1h is None:
        raise ValueError("기온(T1H)을 가져오지 못했습니다. nx/ny 또는 base_time을 확인하세요.")
    if rn1 is None:
        rn1 = 0.0

    return {
        "today_ymd": now.strftime("%Y%m%d"),
        "temp": t1h,
        "rain": rn1,
        "base_date": base_date,
        "base_time": base_time,
    }

In [8]:
# -------------------------
# 4) 학습 데이터 만들기: 시간/동 단위 데이터 -> 일별 매출(AMT 합)
#    날씨도 일별로 집계(평균기온, 일강수량합)
# -------------------------
def build_daily_dataset(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["TA_YMD"] = df["TA_YMD"].astype(str)
    # 일별 집계
    daily = df.groupby("TA_YMD", as_index=False).agg(
        AMT_sum=("AMT", "sum"),
        TEMP_mean=("TEMP", "mean"),
        RAIN_sum=("RAIN", "sum"),
        DAY_mode=("DAY", lambda x: int(pd.Series(x).mode().iloc[0]) if len(pd.Series(x).mode()) else int(x.iloc[0])),
        # 보조 피처(선택): CNT 합, UNIT 평균 등도 추가 가능
        CNT_sum=("CNT", "sum"),
        UNIT_mean=("UNIT", "mean"),
    )
    # 날짜 파생
    daily["date"] = pd.to_datetime(daily["TA_YMD"], format="%Y%m%d")
    daily["year"] = daily["date"].dt.year
    daily["month"] = daily["date"].dt.month
    daily["day"] = daily["date"].dt.day
    return daily


In [9]:
# -------------------------
# 5) 모델 학습 (일별 AMT 예측)
#    - 범주형: DAY_mode(요일코드)
#    - 수치형: TEMP_mean, RAIN_sum, month, (선택: CNT_sum, UNIT_mean 등)
# -------------------------
def train_sales_model(daily: pd.DataFrame):
    # 타겟: 일매출(AMT 합)
    y = daily["AMT_sum"].values

    # 입력 피처 구성(필요 최소)
    X = daily[["DAY_mode", "TEMP_mean", "RAIN_sum", "month"]].copy()

    cat_cols = ["DAY_mode"]
    num_cols = ["TEMP_mean", "RAIN_sum", "month"]

    pre = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
            ("num", "passthrough", num_cols),
        ]
    )

    model = Pipeline(
        steps=[
            ("preprocess", pre),
            ("rf", RandomForestRegressor(
                n_estimators=600,
                random_state=42,
                max_depth=12,
                min_samples_leaf=2,
                n_jobs=-1
            )),
        ]
    )

    # 시간순 분할(가장 최근 20%를 테스트로)
    daily_sorted = daily.sort_values("date").reset_index(drop=True)
    X_sorted = daily_sorted[["DAY_mode", "TEMP_mean", "RAIN_sum", "month"]]
    y_sorted = daily_sorted["AMT_sum"].values

    split = int(len(daily_sorted) * 0.8)
    X_train, X_test = X_sorted.iloc[:split], X_sorted.iloc[split:]
    y_train, y_test = y_sorted[:split], y_sorted[split:]

    model.fit(X_train, y_train)

    pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, pred)
    r2 = r2_score(y_test, pred)

    return model, {"MAE": mae, "R2": r2, "train_days": split, "test_days": len(daily_sorted) - split}


In [10]:
# -------------------------
# 6) 오늘 매출 예측 실행
# -------------------------
def predict_today_sales():
    # (1) 데이터 로드
    df = pd.read_csv(CSV_PATH)

    # (2) 일별 학습셋 만들기
    daily = build_daily_dataset(df)

    # (3) 모델 학습
    model, metrics = train_sales_model(daily)
    print("[모델 평가(최근 20% 테스트)]", metrics)

    # (4) 오늘 날씨 가져오기(초단기실황)
    w = fetch_today_weather_ultra_ncst(NX, NY, SERVICE_KEY)
    today_dt = datetime.strptime(w["today_ymd"], "%Y%m%d").replace(tzinfo=KST)
    day_code = to_day_code(today_dt)

    # (5) 오늘 입력 피처 구성
    X_today = pd.DataFrame([{
        "DAY_mode": day_code,
        "TEMP_mean": w["temp"],   # 오늘 대표 기온(현재 실황)
        "RAIN_sum": w["rain"],    # 오늘 대표 강수(현재 1시간 강수)
        "month": today_dt.month,
    }])

    # (6) 예측
    yhat = model.predict(X_today)[0]
    print(f"[오늘({w['today_ymd']}) 예측 매출(AMT)] {yhat:,.0f} 원")
    print(f"(기상 API 기준시각: {w['base_date']} {w['base_time']}, temp={w['temp']}°C, rain={w['rain']}mm)")

if __name__ == "__main__":
    predict_today_sales()


[모델 평가(최근 20% 테스트)] {'MAE': 17858319.325807597, 'R2': 0.7243961541733086, 'train_days': 828, 'test_days': 207}
[오늘(20251224) 예측 매출(AMT)] 134,949,970 원
(기상 API 기준시각: 20251224 1500, temp=5.3°C, rain=0.0mm)
